## Problem NetCDF files

The dummy ‘crs’ variable exists to store projection info for the data in
a NetCDF file.

When xarray reads one dataset with `open_dataset()` it correctly sits on
its own unused scalar dimension, but with `open_mfdataset()` crs is on
the ‘time’ dimension. This causes problems with append.

(I cannot get this to work from the github URLs, so I’m just assuming
you download some [from
here](https://github.com/mdsumner/xarray_vrt_expt/tree/master/data/nsidc),
or wget as here).

    wget https://github.com/mdsumner/xarray_vrt_expt/raw/master/data/nsidc/NSIDC0051_SEAICE_PS_S25km_20220522_v2.0.nc
    wget https://github.com/mdsumner/xarray_vrt_expt/raw/master/data/nsidc/NSIDC0051_SEAICE_PS_S25km_20220523_v2.0.nc

### ok with open_dataset

This first example is correct, crs is on its own (not associated with
time dimension).

In [1]:
import xarray as xr 
xr.open_dataset('NSIDC0051_SEAICE_PS_S25km_20220522_v2.0.nc', engine = 'netcdf4')

<xarray.Dataset>
Dimensions: (x: 316, y: 332, time: 1)
Coordinates:
 * x (x) float64 -3.938e+06 -3.912e+06 ... 3.912e+06 3.938e+06
 * y (y) float64 4.338e+06 4.312e+06 ... -3.912e+06 -3.938e+06
 * time (time) datetime64[ns] 2022-05-22
Data variables:
 crs |S1 b''
 F17_ICECON (time, y, x) float32 ...
Attributes: (12/49)
 title: Sea Ice Concentrations from Nimbus-7 SMMR and ...
 summary: This data set is generated from brightness tem...
 id: 10.5067/MPYG15WAA4WX
 license: Access Constraint: These data are freely, open...
 acknowledgment: These data are produced by the NASA Cryospheri...
 metadata_link: https://doi.org/10.5067/MPYG15WAA4WX
 ... ...
 geospatial_lat_units: degrees_north
 geospatial_lon_units: degrees_east
 product_version: v2.0
 source: Polar stereographic brightness temperatures fr...
 instrument: SSMIS > Special Sensor Microwave Imager/Sounder
 platform: DMSP 5D-3/F17 > Defense Meteorological Satelli... xarray.Dataset Dimensions: x : 316 y : 332 time : 1 Coordinates: (3) x (x) float64 -3.938e+06 -3.912e+06 ... 3.938e+06 standard_name : projection_x_coordinate coverage_content_type : coordinate long_name : x short_name : x axis : X units : meters valid_range : [-3950000. 3950000.] array([-3937500., -3912500., -3887500., ..., 3887500., 3912500., 3937500.]) y (y) float64 4.338e+06 4.312e+06 ... -3.938e+06 standard_name : projection_y_coordinate coverage_content_type : coordinate long_name : y short_name : y axis : Y units : meters valid_range : [-3950000. 4350000.] array([ 4337500., 4312500., 4287500., ..., -3887500., -3912500., -3937500.]) time (time) datetime64[ns] 2022-05-22 standard_name : time coverage_content_type : coordinate long_name : ANSI date axis : T array(['2022-05-22T00:00:00.000000000'], dtype='datetime64[ns]') Data variables: (2) crs () |S1 ... grid_mapping_name : polar_stereographic straight_vertical_longitude_from_pole : 0.0 false_easting : 0.0 false_northing : 0.0 latitude_of_projection_origin : -90.0 standard_parallel : -70.0 long_name : NSIDC_SH_PolarStereo_25km longitude_of_prime_meridian : 0.0 semi_major_axis : 6378273.0 inverse_flattening : 298.279411123064 proj4text : +proj=stere +lat_0=-90 +lat_ts=-70 +lon_0=0 +k=1 +x_0=0 +y_0=0 +a=6378273 +b=6356889.449 +units=m +no_defs crs_wkt : PROJCS["NSIDC Sea Ice Polar Stereographic South",GEOGCS["Unspecified datum based upon the Hughes 1980 ellipsoid",DATUM["Not_specified_based_on_Hughes_1980_ellipsoid",SPHEROID["Hughes 1980",6378273,298.279411123061,AUTHORITY["EPSG","7058"]],AUTHORITY["EPSG","6054"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4054"]],PROJECTION["Polar_Stereographic"],PARAMETER["latitude_of_origin",-70],PARAMETER["central_meridian",0],PARAMETER["scale_factor",1],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["X",EAST],AXIS["Y",NORTH],AUTHORITY["EPSG","3412"]] spatial_ref : PROJCS["NSIDC Sea Ice Polar Stereographic South",GEOGCS["Unspecified datum based upon the Hughes 1980 ellipsoid",DATUM["Not_specified_based_on_Hughes_1980_ellipsoid",SPHEROID["Hughes 1980",6378273,298.279411123061,AUTHORITY["EPSG","7058"]],AUTHORITY["EPSG","6054"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4054"]],PROJECTION["Polar_Stereographic"],PARAMETER["latitude_of_origin",-70],PARAMETER["central_meridian",0],PARAMETER["scale_factor",1],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["X",EAST],AXIS["Y",NORTH],AUTHORITY["EPSG","3412"]] srid : urn:ogc:def:crs:EPSG::3412 GeoTransform : -3950000 25000 0 4350000 0 -25000 array(b'', dtype='|S1') F17_ICECON (time, y, x) float32 ... legacy_binary_header : [ 48 48 50 53 53 0 32 32 51 49 54 0 32 32 51 51 50 0
 49 46 55 57 57 0 45 53 49 46 51 0 50 55 48 46 48 0
 53 53 56 46 52 0 49 53 56 46 48 0 49 55 52 46 48 0
 83 83 77 73 83 0 49 55 32 99 110 

### Not ok with open multiple

Oops, crs is on time dimension.

In [2]:
## open two and crs is living on the time dimension
xr.open_mfdataset(['NSIDC0051_SEAICE_PS_S25km_20220522_v2.0.nc', 
                  'NSIDC0051_SEAICE_PS_S25km_20220523_v2.0.nc'], engine = 'netcdf4')

<xarray.Dataset>
Dimensions: (time: 2, x: 316, y: 332)
Coordinates:
 * x (x) float64 -3.938e+06 -3.912e+06 ... 3.912e+06 3.938e+06
 * y (y) float64 4.338e+06 4.312e+06 ... -3.912e+06 -3.938e+06
 * time (time) datetime64[ns] 2022-05-22 2022-05-23
Data variables:
 crs (time) |S1 b'' b''
 F17_ICECON (time, y, x) float32 dask.array<chunksize=(1, 332, 316), meta=np.ndarray>
Attributes: (12/49)
 title: Sea Ice Concentrations from Nimbus-7 SMMR and ...
 summary: This data set is generated from brightness tem...
 id: 10.5067/MPYG15WAA4WX
 license: Access Constraint: These data are freely, open...
 acknowledgment: These data are produced by the NASA Cryospheri...
 metadata_link: https://doi.org/10.5067/MPYG15WAA4WX
 ... ...
 geospatial_lat_units: degrees_north
 geospatial_lon_units: degrees_east
 product_version: v2.0
 source: Polar stereographic brightness temperatures fr...
 instrument: SSMIS > Special Sensor Microwave Imager/Sounder
 platform: DMSP 5D-3/F17 > Defense Meteorological Satelli... xarray.Dataset Dimensions: time : 2 x : 316 y : 332 Coordinates: (3) x (x) float64 -3.938e+06 -3.912e+06 ... 3.938e+06 standard_name : projection_x_coordinate coverage_content_type : coordinate long_name : x short_name : x axis : X units : meters valid_range : [-3950000. 3950000.] array([-3937500., -3912500., -3887500., ..., 3887500., 3912500., 3937500.]) y (y) float64 4.338e+06 4.312e+06 ... -3.938e+06 standard_name : projection_y_coordinate coverage_content_type : coordinate long_name : y short_name : y axis : Y units : meters valid_range : [-3950000. 4350000.] array([ 4337500., 4312500., 4287500., ..., -3887500., -3912500., -3937500.]) time (time) datetime64[ns] 2022-05-22 2022-05-23 standard_name : time coverage_content_type : coordinate long_name : ANSI date axis : T array(['2022-05-22T00:00:00.000000000', '2022-05-23T00:00:00.000000000'],
 dtype='datetime64[ns]') Data variables: (2) crs (time) |S1 b'' b'' grid_mapping_name : polar_stereographic straight_vertical_longitude_from_pole : 0.0 false_easting : 0.0 false_northing : 0.0 latitude_of_projection_origin : -90.0 standard_parallel : -70.0 long_name : NSIDC_SH_PolarStereo_25km longitude_of_prime_meridian : 0.0 semi_major_axis : 6378273.0 inverse_flattening : 298.279411123064 proj4text : +proj=stere +lat_0=-90 +lat_ts=-70 +lon_0=0 +k=1 +x_0=0 +y_0=0 +a=6378273 +b=6356889.449 +units=m +no_defs crs_wkt : PROJCS["NSIDC Sea Ice Polar Stereographic South",GEOGCS["Unspecified datum based upon the Hughes 1980 ellipsoid",DATUM["Not_specified_based_on_Hughes_1980_ellipsoid",SPHEROID["Hughes 1980",6378273,298.279411123061,AUTHORITY["EPSG","7058"]],AUTHORITY["EPSG","6054"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4054"]],PROJECTION["Polar_Stereographic"],PARAMETER["latitude_of_origin",-70],PARAMETER["central_meridian",0],PARAMETER["scale_factor",1],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["X",EAST],AXIS["Y",NORTH],AUTHORITY["EPSG","3412"]] spatial_ref : PROJCS["NSIDC Sea Ice Polar Stereographic South",GEOGCS["Unspecified datum based upon the Hughes 1980 ellipsoid",DATUM["Not_specified_based_on_Hughes_1980_ellipsoid",SPHEROID["Hughes 1980",6378273,298.279411123061,AUTHORITY["EPSG","7058"]],AUTHORITY["EPSG","6054"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4054"]],PROJECTION["Polar_Stereographic"],PARAMETER["latitude_of_origin",-70],PARAMETER["central_meridian",0],PARAMETER["scale_factor",1],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["X",EAST],AXIS["Y",NORTH],AUTHORITY["EPSG","3412"]] srid : urn:ogc:def:crs:EPSG::3412 GeoTransform : -3950000 25000 0 4350000 0 -25000 array([b'', b''], dtype='|S1') F17_ICECON (time, y, x) float32 dask.array<chunksize=(1, 332, 316), meta=np.ndarray> legacy_binary_header : [ 48 48 50 

### Ok when multiple open only one.

Ok when we only use one source.

In [3]:
## open 1 it doesn't happen
xr.open_mfdataset('NSIDC0051_SEAICE_PS_S25km_20220522_v2.0.nc', engine = 'netcdf4')

<xarray.Dataset>
Dimensions: (x: 316, y: 332, time: 1)
Coordinates:
 * x (x) float64 -3.938e+06 -3.912e+06 ... 3.912e+06 3.938e+06
 * y (y) float64 4.338e+06 4.312e+06 ... -3.912e+06 -3.938e+06
 * time (time) datetime64[ns] 2022-05-22
Data variables:
 crs |S1 ...
 F17_ICECON (time, y, x) float32 dask.array<chunksize=(1, 332, 316), meta=np.ndarray>
Attributes: (12/49)
 title: Sea Ice Concentrations from Nimbus-7 SMMR and ...
 summary: This data set is generated from brightness tem...
 id: 10.5067/MPYG15WAA4WX
 license: Access Constraint: These data are freely, open...
 acknowledgment: These data are produced by the NASA Cryospheri...
 metadata_link: https://doi.org/10.5067/MPYG15WAA4WX
 ... ...
 geospatial_lat_units: degrees_north
 geospatial_lon_units: degrees_east
 product_version: v2.0
 source: Polar stereographic brightness temperatures fr...
 instrument: SSMIS > Special Sensor Microwave Imager/Sounder
 platform: DMSP 5D-3/F17 > Defense Meteorological Satelli... xarray.Dataset Dimensions: x : 316 y : 332 time : 1 Coordinates: (3) x (x) float64 -3.938e+06 -3.912e+06 ... 3.938e+06 standard_name : projection_x_coordinate coverage_content_type : coordinate long_name : x short_name : x axis : X units : meters valid_range : [-3950000. 3950000.] array([-3937500., -3912500., -3887500., ..., 3887500., 3912500., 3937500.]) y (y) float64 4.338e+06 4.312e+06 ... -3.938e+06 standard_name : projection_y_coordinate coverage_content_type : coordinate long_name : y short_name : y axis : Y units : meters valid_range : [-3950000. 4350000.] array([ 4337500., 4312500., 4287500., ..., -3887500., -3912500., -3937500.]) time (time) datetime64[ns] 2022-05-22 standard_name : time coverage_content_type : coordinate long_name : ANSI date axis : T array(['2022-05-22T00:00:00.000000000'], dtype='datetime64[ns]') Data variables: (2) crs () |S1 ... grid_mapping_name : polar_stereographic straight_vertical_longitude_from_pole : 0.0 false_easting : 0.0 false_northing : 0.0 latitude_of_projection_origin : -90.0 standard_parallel : -70.0 long_name : NSIDC_SH_PolarStereo_25km longitude_of_prime_meridian : 0.0 semi_major_axis : 6378273.0 inverse_flattening : 298.279411123064 proj4text : +proj=stere +lat_0=-90 +lat_ts=-70 +lon_0=0 +k=1 +x_0=0 +y_0=0 +a=6378273 +b=6356889.449 +units=m +no_defs crs_wkt : PROJCS["NSIDC Sea Ice Polar Stereographic South",GEOGCS["Unspecified datum based upon the Hughes 1980 ellipsoid",DATUM["Not_specified_based_on_Hughes_1980_ellipsoid",SPHEROID["Hughes 1980",6378273,298.279411123061,AUTHORITY["EPSG","7058"]],AUTHORITY["EPSG","6054"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4054"]],PROJECTION["Polar_Stereographic"],PARAMETER["latitude_of_origin",-70],PARAMETER["central_meridian",0],PARAMETER["scale_factor",1],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["X",EAST],AXIS["Y",NORTH],AUTHORITY["EPSG","3412"]] spatial_ref : PROJCS["NSIDC Sea Ice Polar Stereographic South",GEOGCS["Unspecified datum based upon the Hughes 1980 ellipsoid",DATUM["Not_specified_based_on_Hughes_1980_ellipsoid",SPHEROID["Hughes 1980",6378273,298.279411123061,AUTHORITY["EPSG","7058"]],AUTHORITY["EPSG","6054"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4054"]],PROJECTION["Polar_Stereographic"],PARAMETER["latitude_of_origin",-70],PARAMETER["central_meridian",0],PARAMETER["scale_factor",1],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["X",EAST],AXIS["Y",NORTH],AUTHORITY["EPSG","3412"]] srid : urn:ogc:def:crs:EPSG::3412 GeoTransform : -3950000 25000 0 4350000 0 -25000 array(b'', dtype='|S1') F17_ICECON (time, y, x) float32 dask.array<chunksize=(1, 332, 316), meta=np.ndarray> legacy_binary_header : [ 48 48 50 53 53 0 32 32 51 49 54 0 32 32 51 51 50 0
 49 46 55 57 57 0 45 53 49 46 51 0 50 55

In [4]:
xr.open_mfdataset(['NSIDC0051_SEAICE_PS_S25km_20220522_v2.0.nc'], engine = 'netcdf4')

<xarray.Dataset>
Dimensions: (x: 316, y: 332, time: 1)
Coordinates:
 * x (x) float64 -3.938e+06 -3.912e+06 ... 3.912e+06 3.938e+06
 * y (y) float64 4.338e+06 4.312e+06 ... -3.912e+06 -3.938e+06
 * time (time) datetime64[ns] 2022-05-22
Data variables:
 crs |S1 ...
 F17_ICECON (time, y, x) float32 dask.array<chunksize=(1, 332, 316), meta=np.ndarray>
Attributes: (12/49)
 title: Sea Ice Concentrations from Nimbus-7 SMMR and ...
 summary: This data set is generated from brightness tem...
 id: 10.5067/MPYG15WAA4WX
 license: Access Constraint: These data are freely, open...
 acknowledgment: These data are produced by the NASA Cryospheri...
 metadata_link: https://doi.org/10.5067/MPYG15WAA4WX
 ... ...
 geospatial_lat_units: degrees_north
 geospatial_lon_units: degrees_east
 product_version: v2.0
 source: Polar stereographic brightness temperatures fr...
 instrument: SSMIS > Special Sensor Microwave Imager/Sounder
 platform: DMSP 5D-3/F17 > Defense Meteorological Satelli... xarray.Dataset Dimensions: x : 316 y : 332 time : 1 Coordinates: (3) x (x) float64 -3.938e+06 -3.912e+06 ... 3.938e+06 standard_name : projection_x_coordinate coverage_content_type : coordinate long_name : x short_name : x axis : X units : meters valid_range : [-3950000. 3950000.] array([-3937500., -3912500., -3887500., ..., 3887500., 3912500., 3937500.]) y (y) float64 4.338e+06 4.312e+06 ... -3.938e+06 standard_name : projection_y_coordinate coverage_content_type : coordinate long_name : y short_name : y axis : Y units : meters valid_range : [-3950000. 4350000.] array([ 4337500., 4312500., 4287500., ..., -3887500., -3912500., -3937500.]) time (time) datetime64[ns] 2022-05-22 standard_name : time coverage_content_type : coordinate long_name : ANSI date axis : T array(['2022-05-22T00:00:00.000000000'], dtype='datetime64[ns]') Data variables: (2) crs () |S1 ... grid_mapping_name : polar_stereographic straight_vertical_longitude_from_pole : 0.0 false_easting : 0.0 false_northing : 0.0 latitude_of_projection_origin : -90.0 standard_parallel : -70.0 long_name : NSIDC_SH_PolarStereo_25km longitude_of_prime_meridian : 0.0 semi_major_axis : 6378273.0 inverse_flattening : 298.279411123064 proj4text : +proj=stere +lat_0=-90 +lat_ts=-70 +lon_0=0 +k=1 +x_0=0 +y_0=0 +a=6378273 +b=6356889.449 +units=m +no_defs crs_wkt : PROJCS["NSIDC Sea Ice Polar Stereographic South",GEOGCS["Unspecified datum based upon the Hughes 1980 ellipsoid",DATUM["Not_specified_based_on_Hughes_1980_ellipsoid",SPHEROID["Hughes 1980",6378273,298.279411123061,AUTHORITY["EPSG","7058"]],AUTHORITY["EPSG","6054"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4054"]],PROJECTION["Polar_Stereographic"],PARAMETER["latitude_of_origin",-70],PARAMETER["central_meridian",0],PARAMETER["scale_factor",1],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["X",EAST],AXIS["Y",NORTH],AUTHORITY["EPSG","3412"]] spatial_ref : PROJCS["NSIDC Sea Ice Polar Stereographic South",GEOGCS["Unspecified datum based upon the Hughes 1980 ellipsoid",DATUM["Not_specified_based_on_Hughes_1980_ellipsoid",SPHEROID["Hughes 1980",6378273,298.279411123061,AUTHORITY["EPSG","7058"]],AUTHORITY["EPSG","6054"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4054"]],PROJECTION["Polar_Stereographic"],PARAMETER["latitude_of_origin",-70],PARAMETER["central_meridian",0],PARAMETER["scale_factor",1],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["X",EAST],AXIS["Y",NORTH],AUTHORITY["EPSG","3412"]] srid : urn:ogc:def:crs:EPSG::3412 GeoTransform : -3950000 25000 0 4350000 0 -25000 array(b'', dtype='|S1') F17_ICECON (time, y, x) float32 dask.array<chunksize=(1, 332, 316), meta=np.ndarray> legacy_binary_header : [ 48 48 50 53 53 0 32 32 51 49 54 0 32 32 51 51 50 0
 49 46 55 57 57 0 45 53 49 46 51 0 50 55